## 8-1. 하이브리드 추천 시스템

***하이브리드(hybrid)추천 알고리즘 or 하이브리드 추천 시스템***  
***: 복수의 추천 알고리즘을 결합해서 추천하는 알고리즘***

하이브리드 추천 알고리즘은 **하나의 알고리즘만 사용하는 경우보다 더 정확한 경우가 많다!**  
그 이유는 복수의 알고리즘이 결합되는 경우에, 한 알고리즘은 다른 알고리즘의 오류를 보정하는 역할을 하는 경우가 많기 때문이다.  

하지만 복수의 알고리즘의 결합이 항상 좋은 결과를 가져오는 것은 아니기 때문에 **실제 결합의 효과가 있는지를 확인해보고 결합**해야 할 것이다.  
또한 **복수의 알고리즘을 어떻게 결합하는가**에 따라 성능이 많이 좌우되기도 하기 때문에 최적의 결합 방법을 찾기 위한 다양한 분석이 선행되어야 한다.


## 8-2. 하이브리드 추천 시스템의 원리

가상의 추천엔진 2개를 결합하는 코드

In [1]:
# Hybrid 추천 - Dummy engine 사용
##### (1): train/test set 분리, RMSE 계산

import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle

# csv 파일에서 불러오기
r_cols  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

# train test 분리
TRAIN_SIZE    = 0.75
ratings       = shuffle(ratings, random_state=1)  # random_state로 랜덤시드 설정 - 같은 시드 같은 순서
cutoff        = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test  = ratings.iloc[cutoff:]

# RMSE 계산을 위한 함수
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))



##### (2): 두 개의 추천엔진에 각각 0.8 0.2의 가중치 부여하는 예를 설명

# Dummy recommender 0 - 추천 엔진 흉내내기 (임의의 수 반환)
def recommender0(recomm_list):
    recommendations = []
    for pair in recomm_list:
        recommendations.append(random.random() * 4 + 1)
    return np.array(recommendations)

# Dummy recommender 1 - 추천 엔진 흉내내기 (임의의 수 반환)
def recommender1(recomm_list):
    recommendations = []
    for pair in recomm_list:
        recommendations.append(random.random() * 4 + 1)
    return np.array(recommendations)

# Hybrid 결과 얻기
weight       = [0.8, 0.2]                   # 두 추천 엔진의 결합 weight
recomm_list  = np.array(ratings_test)
predictions0 = recommender0(recomm_list)    # 첫 엔진의 예측값
predictions1 = recommender1(recomm_list)    # 두번째 엔진의 예측값
predictions  = predictions0 * weight[0] + predictions1 * weight[1]
RMSE2(recomm_list[:, 2], predictions)



1.5660515352198248

하이브리드 추천엔진의 원리를 설명하기 위해서 랜덤 숫자를 예측치로 사용했기 때문에 RMSE값이 아주 나쁘게 나온다.

## 8-3. 하이브리드 추천 시스템 (CF와 MF의 결합)

In [9]:
# Hybrid 추천 - CF + MF

import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# 데이터 읽어 오기 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

# train test 분리
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

# 정확도(RMSE)를 계산하는 함수 
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [17]:
##### CF 추천 알고리즘 >>>>>>>>>>>>>>>

rating_matrix = ratings_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set 사용자들의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy    = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# train 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)    # user별 평점평균
rating_bias = (rating_matrix.T - rating_mean).T

def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores      = user_similarity[user_id]
        movie_ratings   = rating_bias[movie_id]
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings   = movie_ratings.drop(none_rating_idx)
        sim_scores      = sim_scores.drop(none_rating_idx)
        if neighbor_size == 0:  # 해당 영화 rating한 모든 유저 기반
            prediction     = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction     = prediction + rating_mean[user_id]  # 예상편차 + 평균
        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores    = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)     
                user_idx      = np.argsort(sim_scores)      # 오름차순 정렬된 인덱스 반환
                sim_scores    = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction    = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction    = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [18]:

##### MF 추천 알고리즘 >>>>>>>>>>>>>>>

class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)        
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i, 0]]
            y = self.item_id_index[ratings_test.iloc[i, 1]]
            z = ratings_test.iloc[i, 2]
            test_set.append([x, y, z])
            self.R[x, y] = 0                    # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                         # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        prediction = self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
        return prediction

    # Full user-movie rating matrix
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)

# MF클래스 생성 및 학습
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=250, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 10 ; Train RMSE = 0.9664 ; Test RMSE = 0.9834
Iteration: 20 ; Train RMSE = 0.9420 ; Test RMSE = 0.9645
Iteration: 30 ; Train RMSE = 0.9313 ; Test RMSE = 0.9566
Iteration: 40 ; Train RMSE = 0.9253 ; Test RMSE = 0.9524
Iteration: 50 ; Train RMSE = 0.9214 ; Test RMSE = 0.9497
Iteration: 60 ; Train RMSE = 0.9186 ; Test RMSE = 0.9480
Iteration: 70 ; Train RMSE = 0.9165 ; Test RMSE = 0.9468
Iteration: 80 ; Train RMSE = 0.9148 ; Test RMSE = 0.9459
Iteration: 90 ; Train RMSE = 0.9131 ; Test RMSE = 0.9452
Iteration: 100 ; Train RMSE = 0.9113 ; Test RMSE = 0.9445
Iteration: 110 ; Train RMSE = 0.9091 ; Test RMSE = 0.9437
Iteration: 120 ; Train RMSE = 0.9062 ; Test RMSE = 0.9427
Iteration: 130 ; Train RMSE = 0.9019 ; Test RMSE = 0.9412
Iteration: 140 ; Train RMSE = 0.8960 ; Test RMSE = 0.9390
Iteration: 150 ; Train RMSE = 0.8877 ; Test RMSE = 0.9361
Iteration: 160 ; Train RMSE = 0.8772 ; Test RMSE = 0.9324
Iteration: 170 ; Train RMSE = 0.8648 ; Test RMSE = 0.9285
Iteration: 180 ; Train 

In [20]:
##### Hybrid 추천 알고리즘

def recommender0(recomm_list, mf):                  # MF
    recommendations = np.array([mf.get_one_prediction(user, movie) for (user, movie) in recomm_list])
    return recommendations

def recommender1(recomm_list, neighbor_size=0):     # CF
    recommendations = np.array([CF_knn_bias(user, movie, neighbor_size) for (user, movie) in recomm_list])
    return recommendations

recomm_list = np.array(ratings_test.iloc[:, [0, 1]])    # user_id, movie_id만 추출
predictions0 = recommender0(recomm_list, mf)
print("MF의 RMSE: ", RMSE2(ratings_test.iloc[:, 2], predictions0))     # MF의 RMSE
predictions1 = recommender1(recomm_list, 37)
print("CF의 RMSE: ",RMSE2(ratings_test.iloc[:, 2], predictions1))     # CF의 RMSE

# 결합 가중치로 가중평균
weight = [0.8, 0.2]
predictions = predictions0 * weight[0] + predictions1 * weight[1]
# 하이브리드 알고리즘의 RMSE
print("MF+CF hybrid의 RMSE: ",RMSE2(ratings_test.iloc[:, 2], predictions))

# 최적의 가중치 찾기
for i in np.arange(0, 1, 0.01):
    weight = [i, 1.0 - i]
    predictions = predictions0 * weight[0] + predictions1 * weight[1]
    print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], 
                                                    RMSE2(ratings_test.iloc[:, 2], predictions)))

MF의 RMSE:  0.9096003768648583
CF의 RMSE:  0.9467199341641682
MF+CF hybrid의 RMSE:  0.9092863460200453
Weights - 0.00 : 1.00 ; RMSE = 0.9467199
Weights - 0.01 : 0.99 ; RMSE = 0.9458866
Weights - 0.02 : 0.98 ; RMSE = 0.9450621
Weights - 0.03 : 0.97 ; RMSE = 0.9442463
Weights - 0.04 : 0.96 ; RMSE = 0.9434394
Weights - 0.05 : 0.95 ; RMSE = 0.9426412
Weights - 0.06 : 0.94 ; RMSE = 0.9418519
Weights - 0.07 : 0.93 ; RMSE = 0.9410715
Weights - 0.08 : 0.92 ; RMSE = 0.9403000
Weights - 0.09 : 0.91 ; RMSE = 0.9395374
Weights - 0.10 : 0.90 ; RMSE = 0.9387837
Weights - 0.11 : 0.89 ; RMSE = 0.9380390
Weights - 0.12 : 0.88 ; RMSE = 0.9373033
Weights - 0.13 : 0.87 ; RMSE = 0.9365765
Weights - 0.14 : 0.86 ; RMSE = 0.9358588
Weights - 0.15 : 0.85 ; RMSE = 0.9351501
Weights - 0.16 : 0.84 ; RMSE = 0.9344505
Weights - 0.17 : 0.83 ; RMSE = 0.9337600
Weights - 0.18 : 0.82 ; RMSE = 0.9330785
Weights - 0.19 : 0.81 ; RMSE = 0.9324062
Weights - 0.20 : 0.80 ; RMSE = 0.9317431
Weights - 0.21 : 0.79 ; RMSE = 0.931089

- MF의 RMSE가 0.9096, CF의 RMSErk 0.9467일 때,  
  이 둘을 결합한 하이브리드 추천시스템의 RMSE는 0.9092로 미세하지만 두 알고리즘보다 더 좋은 결과를 보인다.

- 최적의 가중치는 Weights - 0.88 : 0.12로 RMSE = 0.9089384가 나온다.

## +) 하이브리드 추천 시스템 (UBCF + IBCF)

In [21]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
        'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
        'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
        return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [26]:
##### IBCF >>>>>>>>>>>>>>>

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기  
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# 아이템 간의 유사도: train set의 모든 가능한 아이템 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy    = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [27]:
##### UBCF >>>>>>>>>>>>>>>

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set 사용자들의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# train 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)          # user별 평가점수의 평균
rating_bias = (rating_matrix.T - rating_mean).T   # train 데이터의 모든 user가 모든 영화 평점에대한 평점 편차


def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores      = user_similarity[user_id]
        movie_ratings   = rating_bias[movie_id]
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings   = movie_ratings.drop(none_rating_idx)
        sim_scores      = sim_scores.drop(none_rating_idx)
        if neighbor_size == 0:  # 해당 영화 rating한 모든 유저 기반
            prediction     = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction     = prediction + rating_mean[user_id]  # 예상편차 + 평균
        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores    = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)     
                user_idx      = np.argsort(sim_scores)      # 오름차순 정렬된 인덱스 반환
                sim_scores    = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction    = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction    = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

# 최소 신뢰도 (공통 평가 영화 수)
SIG_LEVEL = 3
# 최소 사용자 수 (현재 영화를 평가한 사용자 수)
MIN_RATINGS = 2

In [30]:
##### Hybrid 추천 알고리즘

def recommender0(recomm_list)   :                   # IBCF-
    recommendations = np.array([CF_IBCF(user, movie) for (user, movie) in recomm_list])
    return recommendations

def recommender1(recomm_list, neighbor_size=0):     # UBCF-
    recommendations = np.array([CF_knn_bias(user, movie, neighbor_size) for (user, movie) in recomm_list])
    return recommendations

recomm_list = np.array(ratings_test.iloc[:, [0, 1]])
predictions0 = recommender0(recomm_list)
print("IBCF의 RMSE: ", RMSE(ratings_test.iloc[:, 2], predictions0))
predictions1 = recommender1(recomm_list, 37)
print("UBCF의 RMSE: ", RMSE(ratings_test.iloc[:, 2], predictions1))

weight = [0.04, 0.96]
predictions = predictions0 * weight[0] + predictions1 * weight[1]
print("UBCF+IBCF의 RMSE: ",RMSE(ratings_test.iloc[:, 2], predictions))

for i in np.arange(0, 1, 0.01):
    weight = [i, 1.0 - i]
    predictions = predictions0 * weight[0] + predictions1 * weight[1]
    print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], 
                                                    RMSE(ratings_test.iloc[:, 2], predictions)))

IBCF의 RMSE:  0.969185777075798
UBCF의 RMSE:  0.8590448203920592
UBCF+IBCF의 RMSE:  0.8588028997232506
Weights - 0.00 : 1.00 ; RMSE = 0.8590448
Weights - 0.01 : 0.99 ; RMSE = 0.8589458
Weights - 0.02 : 0.98 ; RMSE = 0.8588725
Weights - 0.03 : 0.97 ; RMSE = 0.8588249
Weights - 0.04 : 0.96 ; RMSE = 0.8588029
Weights - 0.05 : 0.95 ; RMSE = 0.8588066
Weights - 0.06 : 0.94 ; RMSE = 0.8588360
Weights - 0.07 : 0.93 ; RMSE = 0.8588911
Weights - 0.08 : 0.92 ; RMSE = 0.8589719
Weights - 0.09 : 0.91 ; RMSE = 0.8590783
Weights - 0.10 : 0.90 ; RMSE = 0.8592104
Weights - 0.11 : 0.89 ; RMSE = 0.8593681
Weights - 0.12 : 0.88 ; RMSE = 0.8595515
Weights - 0.13 : 0.87 ; RMSE = 0.8597605
Weights - 0.14 : 0.86 ; RMSE = 0.8599951
Weights - 0.15 : 0.85 ; RMSE = 0.8602552
Weights - 0.16 : 0.84 ; RMSE = 0.8605409
Weights - 0.17 : 0.83 ; RMSE = 0.8608522
Weights - 0.18 : 0.82 ; RMSE = 0.8611889
Weights - 0.19 : 0.81 ; RMSE = 0.8615512
Weights - 0.20 : 0.80 ; RMSE = 0.8619388
Weights - 0.21 : 0.79 ; RMSE = 0.862351